In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import numpy as np
import os
import re
import string
import random
from transformers import AutoTokenizer

2023-03-20 00:58:30.722743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 00:58:31.910721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64::/home/drake/Documents/projects/IRIS/.conda/lib/
2023-03-20 00:58:31.910848: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64::/home/drake/Documents/proj

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')

2023-03-20 00:58:33.873096: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 00:58:33.891654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 00:58:33.891855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [85]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)


In [86]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [117]:
vocab_size = 30000  # Only consider the top 20k words
maxlen = 80  # Max sequence size
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model


In [119]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", max_vocab_size=vocab_size)

path = os.getcwd()
dir = path.replace('Transformer', 'WikipediaScraper')
os.chdir(dir)
with open (".//data/Chess.txt", "r") as file:
    data = file.readlines()
os.chdir(path)

data = [string for string in data if string != '\n']

#tokenize data
tokenized_data = [None] * len(data)
for i in range(len(data)):
    tokenized_data[i] = tokenizer.encode(data[i], max_length=maxlen + 1, padding='max_length', truncation=True)


#get token vocabs
vocab = tokenizer.get_vocab()

subword_list = [None] * len(vocab)

for subword, index in vocab.items():
    subword_list[index] = subword
vocab = subword_list
def prepare_lm_inputs_labels(text):

    text = tf.cast(text, tf.int64)
    text = tf.convert_to_tensor(text, dtype=tf.int64)
    x = text[:, :-1]
    y = text[:, 1:]
    return x, y

batch_size = 8

text_ds = tf.data.Dataset.from_tensor_slices(tokenized_data)
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)
text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

In [171]:
for x, y in text_ds.take(1):
    print("Sequence shape: ", x.shape)
    print("targets shape: ", y.shape)
    print("Inputs: ", x[0].numpy())
    print("Targets: ", y[0].numpy())

Sequence shape:  (8, 80)
targets shape:  (8, 80)
Inputs:  [  101  1109  5041  6685  7455  1106  1103  7564  1104 10924  2236  1121
  1103  2150  1104  1103  4766  1432   119  2677  1132  1637  1107 19585
  8495 21704   113  3089  3886   114  1105  1141   117  1103 11679  1733
  2328  7147 15662   117  1110  1107 11399   119  1448  1104  1292  6685
   117  1103 24705  4487  2118   118  9468  1918  1377   117  5149  1141
  1104  1103  5041  1637  5756  1104 10924   119  1109 12790  9326 24959
  1403  3263  8167  7155  1115 24705  4487  2118]
Targets:  [ 1109  5041  6685  7455  1106  1103  7564  1104 10924  2236  1121  1103
  2150  1104  1103  4766  1432   119  2677  1132  1637  1107 19585  8495
 21704   113  3089  3886   114  1105  1141   117  1103 11679  1733  2328
  7147 15662   117  1110  1107 11399   119  1448  1104  1292  6685   117
  1103 24705  4487  2118   118  9468  1918  1377   117  5149  1141  1104
  1103  5041  1637  5756  1104 10924   119  1109 12790  9326 24959  1403
  3263

In [120]:
class TextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=10
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x, verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")


# Tokenize starting prompt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

start_prompt = "Chess is"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40

text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)


In [170]:
model = create_model()

if os.path.exists('transformer.h5'):
    model.load_weights('transformer.h5')

transformer = model.fit(text_ds, verbose=1, epochs=500, callbacks=[text_gen_callback])

Epoch 1/500
18/18 [==============================] - 5s 106ms/step - loss: 0.0682 - dense_35_loss: 0.0682
Epoch 2/500
18/18 [==============================] - 2s 105ms/step - loss: 0.0659 - dense_35_loss: 0.0659
Epoch 3/500
18/18 [==============================] - 2s 109ms/step - loss: 0.0647 - dense_35_loss: 0.0647
Epoch 4/500
18/18 [==============================] - 2s 83ms/step - loss: 0.0643 - dense_35_loss: 0.0643
Epoch 5/500
18/18 [==============================] - 1s 72ms/step - loss: 0.0645 - dense_35_loss: 0.0645
Epoch 6/500
18/18 [==============================] - 1s 65ms/step - loss: 0.0646 - dense_35_loss: 0.0646
Epoch 7/500
18/18 [==============================] - 1s 57ms/step - loss: 0.0641 - dense_35_loss: 0.0641
Epoch 8/500
18/18 [==============================] - 1s 53ms/step - loss: 0.0644 - dense_35_loss: 0.0644
Epoch 9/500
18/18 [==============================] - 1s 48ms/step - loss: 0.0642 - dense_35_loss: 0.0642
Epoch 10/500
17/18 [===========================>..] 

KeyboardInterrupt: 

In [167]:
# save the transformer weights
model.save_weights('transformer.h5')

In [168]:
class generate_text():

    def __init__(self, model, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1):
        self.model = model
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k
    
    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)
    
    def detokenize(self, number):
        return self.index_to_word[number]
    
    def generate(self):
        start_tokens = [_ for _ in self.start_tokens]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x, verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            if sample_token == 102:
                break
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = "".join(
            tokenizer.decode(self.start_tokens + tokens_generated)
        )
        print(f"generated text:\n{txt}\n")

In [169]:
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

sentence = "Chess is"
start_tokens = [word_to_index.get(_, 1) for _ in sentence.split()]
print(start_tokens)
max_tokens = 100
text_gen = generate_text(model, max_tokens, start_tokens, vocab)

for i in range(10):
    text_gen.generate()


[11503, 1110]
generated text:
Chess is's international governing body is usually known by its French acronym FIDE ( pronounced FEE - day ) ( French : Fédération internationale des échecs ), or International Chess Federation. FIDE's membership consists of the national chess organizations of over 180 countries ; there are also several associate members, including various supra - national organizations, the International Brail

generated text:
Chess is - playing computer programs ( later known as chess engines ) began to appear in the 1960s. In 1970, the first major computer chess tournament, the North American Computer Chess Championship, was held, followed in 1974 by the first World Computer Chess Championship. In the late 1970s, dedicated home chess computers such as Fidelity Electronics'Chess Challenger became commercially available, as well as software to run on home

generated text:
Chess is's international governing body is usually known by its French acronym FIDE ( pronounced FEE 